[68 Transformer](https://www.bilibili.com/video/BV1Kq4y1H7FL/?spm_id_from=333.788.recommend_more_video.0&vd_source=1d3a7b81d826789081d8b6870d4fff8e)

<img src="http://zh.d2l.ai/_images/transformer.svg">

<img src="picture/截屏2022-06-27 11.03.08.png">

- transformer是完全基于attention的架构, 没有使用到rnn相关内容
### 多头注意力
- <img src="picture/截屏2022-06-27 11.22.06.png">

多头注意力:这个概念有点类似于卷积只中的多通道数, 多头注意力可以理解为创建多个attention层, 每个attention采用单独的全连接层与之对应, 多个attention的输出结合起来, 最后通过全连接层输出. 

<img src="picture/截屏2022-06-27 17.17.26.png">

q k v 都是可学习参数, 头i是第i个attention层, 对应的W均是可学习参数, 多个attention进行contact后进入全新的全连接层,此全连接层也是可学习的参数

<img src="picture/截屏2022-06-27 17.28.49.png">

由于attention没有时间信息, 那么输入的时间，在中间的第i个输出可以看到所有的信息，这个步骤在编码阶段是允许的，但是在解码阶段并不允许，我们可以通过掩码的方式来屏蔽后面的参数，其实就是之前的soft_mask函数

<img src="picture/屏幕截图 2022-06-28 004010.png">

这个浅蓝块块是两个mlp组成，输入维度为：（batch_size批量大小，number_step时间步数或序列长度，hidden_size隐单元数或特征维度）

<img src="picture/屏幕截图 2022-06-28 011405.png">

如果还是用之前的batch_norm来进行参数归一化的话，会出现一个问题，就是在nlp领域，由于使用了soft_mask函数，那么也就是n这个维度会随着vaild_lens的变化而变化，这种情况下，BN的效果并不好，

batch_norm 是在(batch_size,number_step)的矩阵上归一化，layer_norm是以每个(number_step,hidden_size)

<img src="picture/屏幕截图 2022-06-28 013436.png">

这一块还是看代码，看这个具体kv如何来的，以及context如何生成

<img src="picture/屏幕截图 2022-06-28 013656.png">

<img src="picture/屏幕截图 2022-06-28 014030.png">

In [2]:
# 第一步实现多头注意力层
import math
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
# 先不看这部分,直接看多头注意力类
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状
    主要目的就是讲输入的数据按num_heads的个数进行切割,然后交给每个attention层,
    num_hiddens变成了,num_hiddens/num_heads
    """
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1) # 这里的-1对应的是num_hiddens
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，num_hiddens/num_heads)

    # 对调(查询或者“键－值”对的个数，num_heads)->(num_heads, 查询或者“键－值”对的个数)
    X = X.permute(0, 2, 1, 3)
    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数, num_hiddens/num_heads)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)


- 重新回顾点积注意力
- 点积操作要求查询和键具有相同的长度d
- 缩放点积注意力的评分函数为
  - $a(\mathbf q, \mathbf k) = \mathbf{q}^\top \mathbf{k}  /\sqrt{d}.$
- 在实践中，我们通常从小批量的角度来考虑提高效率， 例如基于n个查询和m个键－值对计算注意力， 其中查询和键的长度为d，值的长度为v。 查询$\mathbf Q\in\mathbb R^{n\times d}$、 $\mathbf K\in\mathbb R^{m\times d}$键和$\mathbf V\in\mathbb R^{m\times v}$值的缩放点积注意力是：
  - $$\mathrm{softmax}\left(\frac{\mathbf Q \mathbf K^\top }{\sqrt{d}}\right) \mathbf V \in \mathbb{R}^{n\times v}.$$

In [4]:
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = d2l.masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)


In [5]:
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    # 增加了一个关键参数,num_heads,就是有几个注意力层
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            # 函数详解见附录14
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        #`` output的形状:(batch_size*num_heads，查询的个数，num_hiddens/num_heads)
        # 这里需要解释一个疑问, 就是这样会不会导致每个attention层的参数一致?
        # 不会,因为输入由3维变成了4维,相当于num_heads个attention层参数叠加形成,但是这里输出是三维的
        # 所以加下来需要通过transpose_output将这多个attention一个个拆出来
        output = self.attention(queries, keys, values, valid_lens)
        
        # 由于多个attention本质上是把number_hidden层拆开,所以重新拼接后, num_hidden会还原其长度
        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [6]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

多头注意力输出的形状是（batch_size，num_queries，num_hiddens）。

In [7]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])